In [2]:
pip install https://bitbucket.org/edge-team-leat/microai_public/get/6adfbcb347d3.zip\#subdirectory\=third_party/kerascnn2c_fixed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 4.2 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for kerascnn2c: filename=kerascnn2c-1.0.0-py3-none-any.whl size=21338 sha256=f5a2bed2f449b4f836071713a9769e8919029eecf49dfcc16cc0d2879dcd547f
  Stored in directory: /tmp/pip-ephem-wheel-cache-z8bab21x/wheels/29/df/9b/d62a64e871a29555dc13bc0c189d46297cdf80a3332230aaa1
Successfully built kerascnn2c
Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
from pathlib import Path

import numpy as np
import kerascnn2c
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile

# Read in train directories for cats and dogs before combining them

dirname = 'inputs/cats_dogs/train/dog/'
train_dog = []
for filename in os.listdir(dirname):
    sample_rate, samples = wavfile.read(dirname+filename)
    frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)
    train_dog.append(spectrogram)
    
dirname = 'inputs/cats_dogs/train/cat/'
train_cat = []
for filename in os.listdir(dirname):
    sample_rate, samples = wavfile.read(dirname+filename)
    frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)
    train_cat.append(spectrogram)
    
    

In [6]:
dirname = 'inputs/cats_dogs/test/test/'
test_dog = []
for filename in os.listdir(dirname):
    sample_rate, samples = wavfile.read(dirname+filename)
    frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)
    test_dog.append(spectrogram)
    
dirname = 'inputs/cats_dogs/test/cats/'
test_cat = []
for filename in os.listdir(dirname):
    sample_rate, samples = wavfile.read(dirname+filename)
    frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)
    test_cat.append(spectrogram)

In [7]:
# pad clips so they're all of equal length
def pad_spec(spectrogram, max_length):
    return np.pad(spectrogram, ((0,0),(0,max_length-spectrogram.shape[1])), mode='mean')

max_length = max([s.shape[1] for s in train_dog]+[s.shape[1] for s in train_cat]+[s.shape[1] for s in test_dog]+[s.shape[1] for s in test_cat])

In [8]:
# Aggregate into train and test sets

Y_train = np.array([1]*len(train_dog)+[0]*len(train_cat))
X_train = np.array([pad_spec(s, max_length) for s in train_dog]+[pad_spec(s, max_length) for s in train_cat])

Y_test = np.array([1]*len(test_dog)+[0]*len(test_cat))
X_test = np.array([pad_spec(s, max_length) for s in test_dog]+[pad_spec(s, max_length) for s in test_cat])

In [9]:
# Shuffle train and test sets

p = np.random.permutation(X_train.shape[0])
X_train = X_train[p]
Y_train = Y_train[p]

p = np.random.permutation(X_test.shape[0])
X_test = X_test[p]
Y_test = Y_test[p]

In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models, backend, optimizers

In [55]:
model=tf.keras.Sequential()
model.add(layers.Dense(input_shape=(100,), units= 200,activation='relu'))
model.add(layers.Dense(200,activation='relu'))
model.add(layers.Dense(200,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [56]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [57]:
history = model.fit(X_train, Y_train, epochs=50, batch_size=16,
                    validation_data=(X_test, Y_test))

Epoch 1/50


ValueError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "/opt/conda/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/input_spec.py", line 295, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of layer "sequenti

In [50]:
import copy

model.save('cat_dogs.h5')

model = tf.keras.Model(model.input, model.layers[-1].output, name=model.name)

res = kerascnn2c.Converter(output_path=Path('gsc_output_fixed'),
                           fixed_point=9, # Number of bits for the fractional part, Q7.9 format
                           number_type='int16_t', # Data type for weights/activations (16 bits quantization)
                           long_number_type='int32_t', # Data type for intermediate results
                           number_min=-(2**15), # Minimum value for 16-bit signed integers
                           number_max=(2**15)-1 # Maximum value for 16-bit signed integers
                          ).convert_model(copy.deepcopy(model))
with open('gsc_model_fixed.h', 'w') as f:
    f.write(res)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......conv1d
.........vars
............0
............1
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......flatten
.........vars
......input_layer
.........vars
......max_pooling1d
.........vars
...vars
Keras model archive saving:
File Name                                             Modified             Size
metadata.json                                  2023-04-30 18:09:17           64
config.json                                    2023-04-30 18:09:17         3098
variables.h5                                   2023-04-30 18:09:17       160240
Keras model archive loading:
File Name                                             Modified             Size
metadata.json                                  2023-04-30 18:09:16           64
config.json                                    2023-04-30 18:09:

SystemExit: 1

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [58]:
!g++ -Wall -Wextra -pedantic -Ofast -o gsc_fixed -I gsc_output_fixed/ gsc_output_fixed/model.c main.cpp

In file included from gsc_output_fixed/model.c:15:
gsc_output_fixed/conv1d.c:20:29: error: ‘s’ was not declared in this scope
   20 | #define ZEROPADDING_LEFT    s
      |                             ^
gsc_output_fixed/conv1d.c:23:69: note: in expansion of macro ‘ZEROPADDING_LEFT’
   23 | CONV_OUTSAMPLES     ( ( (INPUT_SAMPLES - CONV_KERNEL_SIZE + ZEROPADDING_LEFT + ZEROPADDING_RIGHT) / CONV_STRIDE ) + 1 )
      |                                                             ^~~~~~~~~~~~~~~~

gsc_output_fixed/conv1d.c:27:51: note: in expansion of macro ‘CONV_OUTSAMPLES’
   27 | typedef number_t conv1d_output_type[CONV_FILTERS][CONV_OUTSAMPLES];
      |                                                   ^~~~~~~~~~~~~~~
gsc_output_fixed/conv1d.c:21:29: error: ‘a’ was not declared in this scope
   21 | #define ZEROPADDING_RIGHT   a
      |                             ^
gsc_output_fixed/conv1d.c:23:88: note: in expansion of macro ‘ZEROPADDING_RIGHT’
   23 |  ( ( (INPUT_SAMPLES - CONV_KERNEL_S

In [ ]:
!./gsc_fixed x_test_gsc_250.csv y_test_gsc_250.csv